### 가진 데이터셋 imp, exp 분류를 위한 Model_extra 만들기 및 위협대화를 exp,imp로 구분


Model_extra : 일반적인 대화 문맥 데이터와 /명시적 언어폭력만을 학습해서,{dktc로는 학습을 하지 않음}  
우리가 가진 dktc 데이터 중 exp와 imp를 분리한다.  

모델1, 2와의 차이점? : 모델 1이 우리가 가진 dktc데이터 중의 exp만으로 학습을 해야 하기 때문에  
dktc 데이터를 exp, imp로 분류하는 모델이 model_extra다. 원리는 모델1과 같음(쓰레스홀딩해서 imp 빼기)  


쓸 데이터셋 : 일상대화문, 비윤리대화문 immoral.csv  


In [ ]:
!pip install keras-bert
!pip install keras-radam
!pip install wandb

# !pip install tensorflow==2.12.0

In [1]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import uuid
# uuid.uuid4() -> UUID('e4392169-4184-43ba-88c9-157d87472c82')

import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense, GlobalMaxPooling1D

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

## 학습에 쓸 데이터 로딩

In [2]:
# daily train & validation
daily_train_df = pd.read_csv("daily_train.csv", usecols=['class','conversation']) #일상대화 train 병합 데이터
daily_val_df = pd.read_csv("daily_val.csv", usecols=['class','conversation']) #일상대화 val 병합 데이터

In [3]:
#immoral.csv -> immoral_extra.csv
immoral_df = pd.read_csv("immoral_extra.csv", usecols=['class','conversation']) #명시적인 위협적 언어폭력만을 모아둠. 모델 extra의 학습할 데이터가 된다.
immoral_df = immoral_df.sample(frac=1).reset_index(drop=True) # 행 섞기


# hate train_validation_split
hate_train_df, hate_val_df = train_test_split(immoral_df, test_size=0.2)

In [4]:
print(daily_train_df.shape)
print(daily_val_df.shape)

(87690, 2)
(10962, 2)


In [5]:
print(hate_train_df.shape)
print(hate_val_df.shape)

(94512, 2)
(23628, 2)


In [4]:
## 테스트를 위한 샘플 만들어두기

daily_train_df_t =daily_train_df.sample(n=50, random_state=1004)
hate_train_df_t =hate_train_df.sample(n=50, random_state=1004)

#몇개만 샘플링

hate_train_df_t['conversation'] = hate_train_df_t['conversation'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


train_df_t = pd.concat([daily_train_df_t, hate_train_df_t], join="inner")
train_df_t.reset_index(drop="index", inplace=True)

In [5]:
daily_train_df =daily_train_df.sample(n=50, random_state=1004)
daily_val_df =daily_val_df.sample(n=10, random_state=1004)

hate_train_df =hate_train_df.sample(n=50, random_state=1004)
hate_val_df =hate_val_df.sample(n=10, random_state=1004)
#몇개만 샘플링

hate_train_df['conversation'] = hate_train_df['conversation'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
hate_val_df['conversation'] = hate_val_df['conversation'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
#특수문자 제거

In [6]:
train_df = pd.concat([daily_train_df, hate_train_df], join="inner")
train_df.reset_index(drop="index", inplace=True)

In [7]:
val_df = pd.concat([daily_val_df, hate_val_df], join="inner")
val_df.reset_index(drop="index", inplace=True)

In [8]:
for i in range(10):
    train_df = train_df.sample(frac=1).reset_index(drop=True) #많이 섞기

for i in range(10):
    val_df = val_df.sample(frac=1).reset_index(drop=True)#많이 섞기

In [12]:
train_df, val_df #잘 섞여지고 인덱스가 정렬되었는지 확인

(   class                                       conversation
 0   일상대화  요즘 불임부부도 많대.\n맞아 공해와 스트레스 때문이라네\n맞아 주변에도 쉽게 찾아...
 1   일상대화  네가 추천해준 가비지타임이라는 웹툰 정말 재미있더라\n네가 농구 좋아해서 그것도 좋...
 2   일상대화  한식 자격증이나 한번 심심한데 따볼까?\n한식은 이미 잘하니까 다른거 따보자.\n그...
 3   위협대화  너는 거지 인증 하는 것도 아니고 또 돈뜯을 궁리만 하냐\n거지도 너만큼 돈 뜯어내...
 4   위협대화  아 우리 회사 밥 너무 개같이 나옴\n그런 폐급 회사 다니면서 뭔 밥이 잘나오길 기...
 ..   ...                                                ...
 95  일상대화  * 벌써 백신 70% 맞았대\n와우 우리가 그럼 삼십퍼\n그래 어쩐지 주위에 다 맞...
 96  위협대화  백종원 선생님 대단하다 다 잘하시네\n근데 안 피곤할까 방송 요리 육아 사업 다 하...
 97  위협대화  bts 요즘 한국팬들한테 소홀하더라 미국 아이돌인줄\n한국 내한 좀 해주세요 오빠들...
 98  일상대화  미용실 자주 가는 편?\n나 머리 잘랐는데 망망\n미용사 미워,...\n나는 삼 사...
 99  위협대화  지하철에 두 자리를 차지하고 앉은 사람이 있다\n그럴 땐 기선제압을 해야해 날 따라...
 
 [100 rows x 2 columns],
    class                                       conversation
 0   일상대화  오늘도 피부에 트러블이 두 개나 더 생겼지 ㅠ\nㅠㅠ 환절기에는 어쩔 수 없는 것 ...
 1   일상대화  출퇴근 너무 힘들어 사람 너무 많아 ㅜㅜ\n아 진짜 오늘도 너무 많았어 키키\n요새...
 2   위협대화  어제 그 드라마 봤어\n응 나도 봤지\n난 주인공이 도대체 왜 그러는지 이해가 안 .

In [9]:
train_df["class"] = train_df["class"].astype('category')
train_df["class"] = train_df["class"].cat.codes # 0:위협대화_hate_and_offensive, 1:일상대화

val_df["class"] = val_df["class"].astype('category')
val_df["class"] = val_df["class"].cat.codes # 0:위협대화_hate_and_offensive, 1:일상대화

In [12]:
train_df['class'].value_counts()

0    50
1    50
Name: class, dtype: int64

In [15]:
train_df

,class,conversation
0,1,요즘 불임부부도 많대.\n맞아 공해와 스트레스 때문이라네\n맞아 주변에도 쉽게 찾아...
1,1,네가 추천해준 가비지타임이라는 웹툰 정말 재미있더라\n네가 농구 좋아해서 그것도 좋...
2,1,한식 자격증이나 한번 심심한데 따볼까?\n한식은 이미 잘하니까 다른거 따보자.\n그...
3,0,너는 거지 인증 하는 것도 아니고 또 돈뜯을 궁리만 하냐\n거지도 너만큼 돈 뜯어내...
4,0,아 우리 회사 밥 너무 개같이 나옴\n그런 폐급 회사 다니면서 뭔 밥이 잘나오길 기...
...,...,...
95,1,* 벌써 백신 70% 맞았대\n와우 우리가 그럼 삼십퍼\n그래 어쩐지 주위에 다 맞...
96,0,백종원 선생님 대단하다 다 잘하시네\n근데 안 피곤할까 방송 요리 육아 사업 다 하...
97,0,bts 요즘 한국팬들한테 소홀하더라 미국 아이돌인줄\n한국 내한 좀 해주세요 오빠들...
98,1,"미용실 자주 가는 편?\n나 머리 잘랐는데 망망\n미용사 미워,...\n나는 삼 사..."


### WandbCallback's labels

In [ ]:
CLASS_NAMES = ["위협 대화", "일상 대화"] # 0 위협, 1 일상

## korBERT 불러와서 model_extra 빌딩

In [10]:
SEQ_LEN = 256

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

DATA_COLUMN = "conversation"
LABEL_COLUMN = "class"

### tokenize

In [11]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
            token = token.replace("_","")
            token = "##" + token
        token_dict[token] = len(token_dict)
        #key(문자) = value(index)

In [12]:
class inherit_Tokenizer(Tokenizer):
    def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [13]:
tokenizer = inherit_Tokenizer(token_dict)
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

### Data encoding

In [14]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN) # conversation
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i]) # class
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str) # conversation


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [15]:
train_x, train_y = load_data(train_df)
val_x, val_y = load_data(val_df)

100%|██████████| 20/20 [00:00<00:00, 1333.51it/s]


In [22]:
train_x

[array([[   101,   9599, 119229, ...,      0,      0,      0],
        [   101,   9011,  11287, ...,      0,      0,      0],
        [   101,   9954,  21155, ...,      0,      0,      0],
        ...,
        [   101,    170,  10806, ...,      0,      0,      0],
        [   101,   9309,  24974, ...,    100,   8926,    102],
        [   101,   9706,  35506, ...,      0,      0,      0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

### 모델 파인튜닝

In [19]:
wandb.login(key="fff4")

wandb: Currently logged in as: m05. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

### wandb.init 에 들어갈 parameter¶

In [20]:
default_config = {
#     "n_channel_1" : 32,
#     "n_channel_2" : 64,
    "n_dense_1" : 256,
    "n_dense_2" : 768,
    "n_dense_3" : 1, 
    "learning_rate" : 1e-4,
    "epochs" : 30,
    "batch_size" : 16,
    "weight_decay" : 0.025,
#     "optimizer" : tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=0.01),
#     "loss" : "categorical_crossentropy",
    "loss" : "binary_crossentropy",
    "metrics" : ["accuracy"]
}

### wandb.sweep의 매개 변수¶

In [21]:
sweep_config = {
#     "name": "test_for_sweep123213", # 변동 가능. 잘 모르겠다 . wandb 페이지에서 어디에 찍히는지...
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
#         "optimizer" : {
#             'value' : 'adam'
#             },
        "batch_size" : {
            "values" : [8, 16, 32]
            },
        "learning_rate" : {
            "min" : 1e-6,
            "max" : 1e-4 # 0에 가까울 수록
            },
        "epochs" : {
            "distribution" : "int_uniform",
            "min" : 15,
            "max" : 30
            }
                    
        }
    }

### model 불러오고 설계

In [22]:
def build_model(n_dense_1, n_dense_2, n_dense_3):
    model = load_trained_model_from_checkpoint(config_path,
                                                checkpoint_path,
# 미리 학습된 모델을 사용하여 새 데이터에 대한 예측을 수행하려는 경우 "training = False
#                                                 training=True, # 새 데이터에서 모델을 계속 훈련 
                                            #     trainable=True,
                                                seq_len=SEQ_LEN)
    # trainable 설정
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Unfreeze the target layer and all subsequent layers
    for layer in model.layers[-1:]:
        layer.trainable = True
        
    inputs = model.inputs[:2]
    dense = model.layers[-1].output


    x = GlobalMaxPooling1D(name="GMP")(dense)
    
    x= tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(n_dense_1, activation='relu', name="1st_dense")(x)
    
    x= tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(n_dense_2, activation='relu', name="2st_dense")(x)
    
    x= tf.keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(n_dense_3, activation='sigmoid', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(x) # 


    model = keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

### checkpoint & earlystopping

In [23]:
path= 'multi'

In [24]:
# sweep 적용하면 ckpt시킨거 overwrited 될 수도 있겠다..
# overwrited 된다면 같은 epoch val_loss 겠지만.......
def keras_callbacks():
    CK = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(path, "extra_sweep.{epoch:02d}-{val_loss:.2f}.h5"),
                                              monitor='val_loss',
                                              save_best_only = True,
                                              save_weights_only = True)

    ES = tf.keras.callbacks.EarlyStopping(patience=1)
    
    return CK, ES

### model.fit 함수

In [25]:
def train():
    global default_config

    # wandb.init & config
    run = wandb.init(config = default_config, 
                     entity = "m05",
                     project = "extra_sweep") # 변동 가능
    config = wandb.config

    # Model -> config.n_channel_1
    model = build_model(config.n_dense_1, config.n_dense_2, config.n_dense_3)    
    
    # model.compile
    model.compile(
                  optimizer = RAdamOptimizer(config.learning_rate, 
                                              config.weight_decay),
                  loss = config.loss,
                  metrics= config.metrics)
    
    #keras_callbacks
    CK, ES = keras_callbacks()
    
    # model.fit
    model.fit(train_x, train_y,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data = (val_x, val_y),
              callbacks = [CK, ES , 
                           WandbCallback(training_data = (val_x[:30], val_y[:30]), # sweep's table에 적용시킨 하이퍼파라미터 수치를 기입이 된다.
                                        labels = CLASS_NAMES)]) # 하지만 아직 사용법 정확히 모름.... 찐 test의 일부분을 넣어야하는디..
                                        
    # model.save ㅠㅠ 저장이 안되었어.........ㅠㅠㅠㅠㅠ
#     wandb.save(os.path.join(path, "wandb.{}.h5".format(str(uuid.uuid4()))))
    
    # evaluate test
    test_loss, test_accuracy = model.evaluate(test_x, test_y, verbose=2)
    
    # wandb's 테이블쪽 column 추가하기
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})
    
    run.finish() # run 종료

### model.fit 실행

In [ ]:
sweep_id = wandb.sweep(sweep_config,
                       entity = 'm05', # 고유명사
                       project = '1st_extra_sweep') # 변동가능

# run the sweep
wandb.agent(sweep_id,
            count=10, # 10회만 돌린다.
            function=train)

Create sweep with ID: gzji46ql
Sweep URL: https://wandb.ai/m05/1st_extra_sweep/sweeps/gzji46ql


wandb: Agent Starting Run: 1fpgp1pl with config:
wandb: 	batch_size: 32
wandb: 	epochs: 27
wandb: 	learning_rate: 8.348607802558546e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/27
4/4 [==============================] - 22s 2s/step - loss: 0.5838 - accuracy: 0.7400 - val_loss: 0.6903 - val_accuracy: 0.5000


INFO:tensorflow:Assets written to: /aiffel/aiffel/DDD/wandb/run-20230411_061733-1fpgp1pl/files/model-best/assets


...

Epoch 23/25
13/13 [==============================] - 3s 199ms/step - loss: 0.3651 - accuracy: 0.9000 - val_loss: 0.2726 - val_accuracy: 1.0000


INFO:tensorflow:Assets written to: /aiffel/aiffel/DDD/wandb/run-20230411_064551-0c04u42g/files/model-best/assets


INFO:tensorflow:Assets written to: /aiffel/aiffel/DDD/wandb/run-20230411_064551-0c04u42g/files/model-best/assets


wandb: Adding directory to artifact (/aiffel/aiffel/DDD/wandb/run-20230411_064551-0c04u42g/files/model-best)... Done. 29.2s


Epoch 24/25
13/13 [==============================] - 3s 199ms/step - loss: 0.3976 - accuracy: 0.8800 - val_loss: 0.2509 - val_accuracy: 1.0000


### Load_weights

In [16]:
# layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    
# 미리 학습된 모델을 사용하여 새 데이터에 대한 예측을 수행하려는 경우 "training = False
#     training=True, # 새 데이터에서 모델을 계속 훈련 
#     trainable=True,
    
    seq_len=SEQ_LEN,) # 최대 문장의 길이 256

In [17]:
def load_model(model):
     # trainable 설정
    for layer in model.layers[:-1]:
        layer.trainable = False

    # Unfreeze the target layer and all subsequent layers
    for layer in model.layers[-1:]:
        layer.trainable = True
    
    
    inputs = model.inputs[:3]
    dense = model.layers[-1].output
    
    x = GlobalMaxPooling1D(name="GMP")(dense)
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(256, activation='relu', name="1st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = keras.layers.Dense(768, activation='relu', name="2st_dense")(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    outputs = keras.layers.Dense(1, activation='sigmoid', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(x) # 


    model = keras.models.Model(inputs=inputs, outputs=outputs)

    model.compile(
                optimizer = RAdamOptimizer(learning_rate=1e-4, 
                                              weight_decay=0.025),
                loss = "binary_crossentropy", 
#                 loss =tf.keras.losses.CategoricalCrossentropy(),
                metrics= ["accuracy"])

    return model

In [18]:
model_extra = load_model(model)

In [21]:
path = 'multi'

In [24]:
model_extra.load_weights(os.path.join(path,"extra_sweep.27-0.48.h5"))

In [25]:
model_extra.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

### Data encoding

In [26]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN) # conversation
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i]) # class
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str) # conversation


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [27]:
train_x_t, train_y_t = load_data(train_df_t)

100%|██████████| 100/100 [00:00<00:00, 1859.63it/s]


In [28]:
tensor = tf.convert_to_tensor(np.array(train_x_t))

---
train_x = tf.convert_to_tensor(train_x)  
train_x[0, :, :],train_x[1, :, :]  
val_x = tf.convert_to_tensor(val_x)  

### predict 1 : train_df 일부분으로 뺀 거

In [29]:
p = model_extra.predict(x=(tensor[0,:,:], tensor[1,:,:]), verbose=1)

4/4 [==============================] - 7s 404ms/step


In [30]:
p

array([[0.8541627 ],
       [0.9133216 ],
       [0.80159545],
       [0.74095505],
       [0.8471909 ],
       [0.8630834 ],
       [0.9127383 ],
       [0.8537537 ],
       [0.81938636],
       [0.8232654 ],
       [0.862426  ],
       [0.8573731 ],
       [0.88749605],
       [0.8830369 ],
       [0.8606315 ],
       [0.8099732 ],
       [0.85034597],
       [0.8437979 ],
       [0.8924318 ],
       [0.8324937 ],
       [0.87852293],
       [0.7911664 ],
       [0.82286054],
       [0.85601586],
       [0.7915448 ],
       [0.8512537 ],
       [0.84664154],
       [0.8757268 ],
       [0.77075696],
       [0.8873754 ],
       [0.8507114 ],
       [0.8124236 ],
       [0.79863274],
       [0.8258251 ],
       [0.68374246],
       [0.8270655 ],
       [0.84216696],
       [0.8073744 ],
       [0.8514049 ],
       [0.85849035],
       [0.80502814],
       [0.81427324],
       [0.87596655],
       [0.8683567 ],
       [0.84250784],
       [0.7340654 ],
       [0.73694533],
       [0.850

In [31]:
train_df_t

,class,conversation
0,일상대화,올해 수능에서도 수험생들 마스크 끼고 시험쳐야한대\n2차백신까지 맞았어도?\n응 예...
1,일상대화,다들 무신사 많이 써?\n나 완전 많이 쓰지 잘 못 사지만\n키키 옷 사기 맨날 실...
2,일상대화,네가 추천해준 가비지타임이라는 웹툰 정말 재미있더라\n네가 농구 좋아해서 그것도 좋...
3,일상대화,나 본캐 바꾸려고\n헐? 왜?\n갑자기 아델이 재미없어졌어\n근데 본캐 바꾸기엔 레...
4,일상대화,"미용실 자주 가는 편?\n나 머리 잘랐는데 망망\n미용사 미워,...\n나는 삼 사..."
...,...,...
95,위협대화,외국인들은 나이가 몇살인지 감이 전혀 안오네\n양키들 10대 후반 지나면 폭삭 늙잔...
96,위협대화,아니 근데 저 여자도 똑같이 더러운인간이면서 왜저렇게 나대는거야\n아니 내말이 진짜...
97,위협대화,인간이 왜 보고 장난이냐\n엄청예쁘구만쩝\n세숫대야 웃긴다\n성격은 어떨까\n으냄새...
98,위협대화,자기 감정 앞세워서 징징대는 애들 너무 싫다\n그런 애들은 아빠한테 골프채로 맞은 ...


### predict 2 : dktc 데이터셋 불러와서 테스트 해보기.

In [32]:
dktc_df = pd.read_csv("DKTC_train.csv", usecols=['class','conversation']) #dktc_train (dktc는 train만 레이블이 달려 있어서 train만 써서 확인해봄)

In [33]:
dktc_df

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [34]:
dktc_x , dktc_y = load_data(dktc_df)

100%|██████████| 3950/3950 [00:02<00:00, 1936.19it/s]


In [35]:
dktc_x

[array([[   101,   9706,  40032, ...,      0,      0,      0],
        [   101,   8934,  18778, ...,      0,      0,      0],
        [   101,   9004,   9098, ...,      0,      0,      0],
        ...,
        [   101, 100585,   9516, ...,      0,      0,      0],
        [   101,   9032,  17196, ...,      0,      0,      0],
        [   101,  10023,  16605, ...,      0,      0,      0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [36]:
tensor_dktc = tf.convert_to_tensor(np.array(dktc_x))

In [37]:
p2 = model_extra.predict(x=(tensor_dktc[0,:,:], tensor_dktc[1,:,:]), verbose=1)

124/124 [==============================] - 75s 601ms/step


In [38]:
np.set_printoptions(threshold=np.inf)

In [39]:
p2

array([[0.81259763],
       [0.78753245],
       [0.8169827 ],
       [0.5499896 ],
       [0.8927549 ],
       [0.8614259 ],
       [0.82730615],
       [0.88624156],
       [0.77648044],
       [0.8152153 ],
       [0.80438155],
       [0.7531689 ],
       [0.87285274],
       [0.79886407],
       [0.82578564],
       [0.7852048 ],
       [0.6828916 ],
       [0.6990942 ],
       [0.6942336 ],
       [0.8341991 ],
       [0.86575425],
       [0.7225031 ],
       [0.67104346],
       [0.8309041 ],
       [0.8868462 ],
       [0.7340534 ],
       [0.8504095 ],
       [0.83627343],
       [0.7192826 ],
       [0.7481024 ],
       [0.8227325 ],
       [0.7961058 ],
       [0.8207298 ],
       [0.78747654],
       [0.68713313],
       [0.66900486],
       [0.76819766],
       [0.7512221 ],
       [0.8225511 ],
       [0.79676205],
       [0.665672  ],
       [0.8205601 ],
       [0.7467665 ],
       [0.8341277 ],
       [0.8285322 ],
       [0.7282777 ],
       [0.8266344 ],
       [0.830

In [40]:
p2.shape

(3950, 1)

In [41]:
indices = []
i=0
for p in p2:
    if p>0.9:
        print(p)
        indices.append(i)
    i+=1
            
    #(0:협박, 1:일상)

[0.91400987]
[0.9141542]
[0.9056739]
[0.9185063]
[0.9011613]
[0.91469574]
[0.90131027]
[0.90007347]
[0.9084721]
[0.90773267]
[0.9026123]
[0.90130824]
[0.9049255]
[0.9013199]
[0.90935695]
[0.90218544]


In [42]:
indices

[604,
 835,
 1219,
 1595,
 1614,
 1927,
 2481,
 2826,
 2896,
 2951,
 3192,
 3444,
 3672,
 3694,
 3831,
 3884]

In [43]:
np.set_printoptions(threshold=10)

In [45]:
dktc_df[['conversation']]

,conversation
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...
3945,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [46]:
for i in indices:
    print(dktc_df[['conversation']].iloc[i])
    print('====================================================')

conversation    고객님 고객님의 정보가 유출되었습니다. 어제부터 모르는 번호로 전화가 오지 않으셨습...
Name: 604, dtype: object
conversation    저 그 이번 주말에 거리에서 집회가 있을 예정입니다. 각 가맹점 마다 사람 세 명씩...
Name: 835, dtype: object
conversation    어떻게 너 같은 애가 우리 오빠랑 결혼을 할 수가 있어?\n선아야. 무슨 말을 그렇...
Name: 1219, dtype: object
conversation    강대리 당신 자리는 오늘부터 저기야\n네.? 저기 몇 년간 사용 안 하던 창고 아닙...
Name: 1595, dtype: object
conversation    오늘 저녁에 회사 업무 끝나고 사장님 생일 때 할 공연 준비해야하는거 알지? 다 남...
Name: 1614, dtype: object
conversation    강인턴 혹시 대학은 뭐 어디 나왔나? 아 이런거 물어보는 거 실례려나\n아유 실례는...
Name: 1927, dtype: object
conversation    민지야 나 요즘 댄스 배우는데 돈이 없어 7만원만 빌려주라.\n응? 7만원?? 나 ...
Name: 2481, dtype: object
conversation    8시에 회의 못한다는게 말이 돼?\n출근시간이 9시인데 어떻게 8시에 해요?\n임원...
Name: 2826, dtype: object
conversation    오는 연휴에 연차 1일 더 써서 조금 길게 쉬고올 수 있을까요?\n안돼. 뭐 땜에 ...
Name: 2896, dtype: object
conversation    와 진짜 쟨 진짜 회사를 차라리 나오지를 말지 일도 못하는게\n네?!\n뭔 대들어 ...
Name: 2951, dtype: object
conversation    저기 우리 애가 아직 어린데 애기 안고 영화보면 저 1명 몫만 영화값 내면 되죠?\...
Name: 3192, dtyp